In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import PIL.Image


In [2]:
tf.__version__

'1.2.0'

In [7]:
import vgg16

In [8]:
vgg16.maybe_download()


- Download progress: 100.0%
Download finished. Extracting files.
Done.
